## Dependencias

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import cufflinks as cf

from datetime import date,time,datetime
from dateutil.relativedelta import relativedelta as rd

from sqlalchemy.engine import create_engine
from sqlalchemy.types import BIGINT,INT,VARCHAR,CHAR,BLOB,DATE,DATETIME,NUMERIC,FLOAT,TEXT,BOOLEAN

import json 
import glob 
import os


cf.go_offline()

## Conexión Base de Datos

In [35]:
creds = json.load(open('creds.json','rb'))
cnx = create_engine(f'mysql+pymysql://{creds["u"]}:{creds["p"]}@{creds["h"]}/{creds["d"]}').connect()
cnx.closed

False

In [36]:
"""Nos conectamos a la tabla intermedia de olap, la cual se creo con sql"""
df = pd.read_sql(con=cnx,sql='select * from olap.intermedia')

In [37]:
df.shape

(21508765, 12)

In [38]:
"""Dimensiones"""
dims = ['mes' ,'sexo' ,'tipo_tarj' ,'marca' ,'d_rng_monto','d_rng_edad','d_estado_com','d_tipo_com'] 

In [39]:
"""Agrupamos por dimensiones y hacemos agregaciones, se resetea el indice para que salga en fomato tabla"""
olap = df.groupby(dims).agg({'id_txn':'count','amount':'sum','is_fraud':'sum'}).reset_index()

In [40]:
olap.shape #dimensiones, se redujo la inicial

(1511507, 11)

In [42]:
olap.head(1)

,mes,sexo,tipo_tarj,marca,d_rng_monto,d_rng_edad,d_estado_com,d_tipo_com,id_txn,amount,is_fraud
0,1991-01-01,M,C,A,00. Reversos,"01. (20,35]",OT,Miscellaneous Food Stores-Convenience Stores a...,2,-186.00,0


In [43]:
olap['d_tipo_com'].map(len).max() #dimension mas grande de d_tipo_com

66

In [44]:
"""Cargamos la información en la tabla de SQL"""
olap.to_sql(con=cnx,chunksize=100000,
name='olap.cubo',
if_exists='replace',
index=False,
dtype=dict(zip(olap.columns,
[DATE,CHAR(1),CHAR(1),CHAR(1),VARCHAR(45),VARCHAR(45),CHAR(2),VARCHAR(66),
BIGINT,NUMERIC(20,2),BIGINT])))

1511507